<a href="https://colab.research.google.com/github/monda00/horse-race-notebook/blob/master/horse_race_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# netkeibaでスクレイピングしたデータのEDA

- 準備
- ライブラリ・データ読み込み
- データの概観・分析

# 準備

## ドメイン知識

- あがり
  - レースや調教の終盤の走破タイム

## データ

- 2019年から2020年までのレースデータ

# ライブラリ・データ読み込み

In [47]:
import numpy as np
import pandas as pd

In [48]:
DATA_PATH = '/content/drive/My Drive/data/horse-race/'

In [50]:
race_df = pd.read_csv(DATA_PATH + 'race.csv')
horse_df = pd.read_csv(DATA_PATH + 'horse.csv')

In [51]:
race_df.head()

,clockwise,date,distance,field_condition,field_type,name,place,race_id,race_round,start_time,weather
0,右,2020/6/22,1400,不,ダ,タイタン賞競走,大井,202044062212,12R,20:50,雨
1,右,2020/6/22,1800,不,ダ,ポートサイド賞競走,大井,202044062211,11R,20:10,雨
2,右,2020/6/22,1200,不,ダ,C1九 十11,大井,202044062210,10R,19:30,雨
3,右,2020/6/22,1400,不,ダ,C1九 十11,大井,202044062209,9R,18:55,雨
4,右,2020/6/22,1600,不,ダ,C1九 十11,大井,202044062208,8R,18:20,雨


In [52]:
horse_df.head()

,agari,age,frame_number,horse_number,horse_weight,jockey,name,popular,race_date,race_id,race_name,rank,time,weight,win
0,38.1,牡3,7.0,13,511(-5),森泰斗,ワイルドホース,2.0,2020/6/22,202044062212,タイタン賞競走,1,1:28.5,54.0,3.7
1,38.1,牝5,6.0,11,475(-2),山崎誠士,クインズオライリー,12.0,2020/6/22,202044062212,タイタン賞競走,2,1:28.7,54.0,81.4
2,38.6,セ6,3.0,4,469(+2),西啓太,メダーリアフレイム,4.0,2020/6/22,202044062212,タイタン賞競走,3,1:28.8,56.0,8.6
3,37.9,牡4,8.0,14,526(+6),藤本現暉,ゴールドプリンス,8.0,2020/6/22,202044062212,タイタン賞競走,4,1:28.8,56.0,34.4
4,38.5,牝5,4.0,7,475(+6),藤田凌,イグレット,9.0,2020/6/22,202044062212,タイタン賞競走,5,1:28.9,54.0,34.8


# データの概観・分析

In [53]:
race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27532 entries, 0 to 27531
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   clockwise        27532 non-null  object
 1   date             27532 non-null  object
 2   distance         27532 non-null  int64 
 3   field_condition  25100 non-null  object
 4   field_type       25100 non-null  object
 5   name             27532 non-null  object
 6   place            27532 non-null  object
 7   race_id          27532 non-null  object
 8   race_round       27532 non-null  object
 9   start_time       26755 non-null  object
 10  weather          26690 non-null  object
dtypes: int64(1), object(10)
memory usage: 2.3+ MB


In [54]:
horse_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293006 entries, 0 to 293005
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   agari         264082 non-null  float64
 1   age           293005 non-null  object 
 2   frame_number  290217 non-null  float64
 3   horse_number  293006 non-null  int64  
 4   horse_weight  293006 non-null  object 
 5   jockey        291874 non-null  object 
 6   name          293003 non-null  object 
 7   popular       289280 non-null  float64
 8   race_date     293006 non-null  object 
 9   race_id       293006 non-null  object 
 10  race_name     293006 non-null  object 
 11  rank          293006 non-null  int64  
 12  time          286445 non-null  object 
 13  weight        293004 non-null  float64
 14  win           293006 non-null  object 
dtypes: float64(4), int64(2), object(9)
memory usage: 33.5+ MB


## データ数

In [55]:
print('race:', race_df.shape[0])
print('horse:', horse_df.shape[0])

race: 27532
horse: 293006


## カラム

In [56]:
race_df.columns

Index(['clockwise', 'date', 'distance', 'field_condition', 'field_type',
       'name', 'place', 'race_id', 'race_round', 'start_time', 'weather'],
      dtype='object')

In [57]:
horse_df.columns

Index(['agari', 'age', 'frame_number', 'horse_number', 'horse_weight',
       'jockey', 'name', 'popular', 'race_date', 'race_id', 'race_name',
       'rank', 'time', 'weight', 'win'],
      dtype='object')

## 欠損値の確認

In [58]:
race_df.isnull().sum()

clockwise             0
date                  0
distance              0
field_condition    2432
field_type         2432
name                  0
place                 0
race_id               0
race_round            0
start_time          777
weather             842
dtype: int64

レース

- field_conditionとfield_typeがないのは障害レースの可能性あり
  - 同じデータ
  - 障害レースにはfield conditionとfield_typeがない
- start_timeがないのはレースが中止？
- weatherがないのは？？

field_conditionとfield_typeが欠損しているデータ

In [59]:
(race_df[race_df['field_condition'].isnull()]['race_id'] == race_df[race_df['field_type'].isnull()]['race_id']).value_counts()

True    2432
Name: race_id, dtype: int64

In [60]:
race_df[race_df['field_condition'].isnull()].head(20)

,clockwise,date,distance,field_condition,field_type,name,place,race_id,race_round,start_time,weather
24,00,2020/6/22,200,NaN,NaN,B3ー6,帯広(ば),202065062211,11R,20:45,曇
25,00,2020/6/22,200,NaN,NaN,第11回ゴールドトロ,帯広(ば),202065062210,10R,20:05,曇
26,00,2020/6/22,200,NaN,NaN,B4ー3,帯広(ば),202065062209,9R,19:25,曇
27,00,2020/6/22,200,NaN,NaN,B4ー7,帯広(ば),202065062208,8R,18:45,曇
28,00,2020/6/22,200,NaN,NaN,B2ー3,帯広(ば),202065062207,7R,18:10,曇
29,00,2020/6/22,200,NaN,NaN,C1ー4,帯広(ば),202065062206,6R,17:35,曇
30,00,2020/6/22,200,NaN,NaN,2歳 未受賞,帯広(ば),202065062205,5R,17:00,曇
31,00,2020/6/22,200,NaN,NaN,2歳 新馬,帯広(ば),202065062204,4R,16:25,曇
32,00,2020/6/22,200,NaN,NaN,2歳 新馬,帯広(ば),202065062203,3R,15:50,曇
33,00,2020/6/22,200,NaN,NaN,2歳 新馬,帯広(ば),202065062202,2R,15:15,曇


In [62]:
race_df[race_df['field_condition'].isnull()]['place'].value_counts()

帯広(ば)    2432
Name: place, dtype: int64

In [16]:
horse_df.isnull().sum()

agari           28924
age                 1
frame_number     2789
horse_number        0
horse_weight        0
jockey           1132
name                3
popular          3726
race_date           0
race_id             0
race_name           0
rank                0
time             6561
weight              2
win                 0
dtype: int64

馬

- age、name、weightは欠損値が極端に少ないので元のサイトを確認する必要あり
- agariはデータがないのが多い（障害レース？）
- frame_number、jockey、popularは？？
- timeがないのはレースに出場しなかった？

## ユニークな値の数

In [63]:
for col, values in race_df.iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))

clockwise: 12
date: 539
distance: 74
field_condition: 19
field_type: 4
name: 9109
place: 51
race_id: 27532
race_round: 12
start_time: 150
weather: 6


In [64]:
for col, values in horse_df.iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))

agari: 264
age: 47
frame_number: 8
horse_number: 25
horse_weight: 18193
jockey: 627
name: 25965
popular: 25
race_date: 539
race_id: 27530
race_name: 9120
rank: 24
time: 2576
weight: 118
win: 5409


- race_idの数がhorse_dfが小さい（２つ足りない）
